# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import os

# File Loading

In [2]:
folder = 'data_cars'
files = os.listdir(folder)

dfs = []

headers = ['URL','Car','Status','Price_1','Price_2','Mileage','Dealer_1','Dealer_2','Dealer_3']

for file in files:
    path = os.path.join(folder, file)
    
    df = pd.read_excel(path, names=headers)
    
    dfs.append(df)

Cars = pd.concat(dfs)

Cars = Cars.drop_duplicates(subset='URL')
Cars.dropna(subset=['Car'], inplace=True)
Cars

,URL,Car,Status,Price_1,Price_2,Mileage,Dealer_1,Dealer_2,Dealer_3
1,https://www.cars.com/vehicledetail/5ee74d2b-dd...,2023 Mazda CX-50 2.5 S Premium Plus Package,New,36703,NaN,"\n\n\n $36,703\n\n MSRP $38,635\n\n\n",NaN,NaN,NaN
2,https://www.cars.com/vehicledetail/460b9155-fa...,2023 Kia Sportage S,New,28990,NaN,"\n\n\n $28,990\n\n\n\n",NaN,Classic Kia,NaN
3,https://www.cars.com/vehicledetail/a0594120-25...,2024 Chevrolet Camaro 2LT,New,41425,NaN,"\n\n\n $41,425\n\n MSRP $41,425\n\n\n",NaN,Classic Chevrolet Beaumont,NaN
4,https://www.cars.com/vehicledetail/b185d19c-b4...,2023 Ford Bronco Badlands,Used,NaN,58900.0,"1,551 mi.",Mike Smith Chrysler Dodge Jeep RAM,NaN,NaN
5,https://www.cars.com/vehicledetail/52bb1768-17...,2021 Acura TLX Advance,Used,NaN,34499.0,"30,384 mi.",Mike Smith Nissan,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10494,https://www.cars.com/vehicledetail/06665049-49...,2023 Chevrolet Corvette Stingray w/3LT,New,122455,NaN,"\n\n\n $122,455\n\n\n\n",NaN,NaN,Chase Chevrolet
10495,https://www.cars.com/vehicledetail/9e367454-56...,2023 Chevrolet Corvette Stingray w/3LT,New,125930,NaN,"\n\n\n $125,930\n\n\n\n",NaN,NaN,Chase Chevrolet
10496,https://www.cars.com/vehicledetail/d578331d-53...,2023 Jeep Wrangler Rubicon,New,53715,NaN,"\n\n\n $53,715\n\n MSRP $58,715\n\n\n ...",NaN,Stockton Dodge Chrysler Jeep RAM,NaN
10497,https://www.cars.com/vehicledetail/3b7dc7d4-60...,2023 Jeep Wrangler Rubicon 392,New,91450,NaN,"\n\n\n $91,450\n\n MSRP $86,455\n\n\n ...",NaN,Stockton Dodge Chrysler Jeep RAM,NaN


Let's navigate into the dataset:

# Data Cleaning

In [3]:
Cars.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80005 entries, 1 to 10498
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URL       80005 non-null  object
 1   Car       80005 non-null  object
 2   Status    80005 non-null  object
 3   Price_1   43680 non-null  object
 4   Price_2   36325 non-null  object
 5   Mileage   80005 non-null  object
 6   Dealer_1  35080 non-null  object
 7   Dealer_2  38574 non-null  object
 8   Dealer_3  6193 non-null   object
dtypes: object(9)
memory usage: 6.1+ MB


In [4]:
Cars.describe()

,URL,Car,Status,Price_1,Price_2,Mileage,Dealer_1,Dealer_2,Dealer_3
count,80005,80005,80005,43680,36325.0,80005,35080,38574,6193
unique,80005,14132,33,22854,16325.0,64573,6852,6685,3175
top,https://www.cars.com/vehicledetail/5ee74d2b-dd...,2023 Ford F-150 XLT,New,Not Priced,12995.0,\n\n\n Not Priced\n\n,Auto Hub,Billion Chrysler Jeep Dodge Ram Fiat of Sioux ...,Elite Motors
freq,1,347,43678,1848,79.0,805,102,91,35


We need to get the next fields:

- Brand
- Model
- Year
- Status
- Mileage
- Dealer
- Price

Let's work on this!

## Brand

In order to get the Brand field, I see two possible approaches:

- Extract from the first words of Car field.
- Use a dictionary.

Personally I prefer the second one, since I suspect the Brand name isn't always in the same position, as well as we will find Brands with one, two, three words that gonna make difficult the task.

In [5]:
brands_df = pd.read_excel('master_data/brands.xlsx')

brands_df

,keyword,brands
0,Ford,Ford
1,Chevrolet,Chevrolet
2,Toyota,Toyota
3,Honda,Honda
4,Nissan,Nissan
...,...,...
58,International Scout,International Scout
59,Geo,Geo
60,Oldsmobile,Oldsmobile
61,Isuzu,Isuzu


In [6]:
brands_dict = brands_df.set_index('keyword')['brands'].to_dict()

def assign_profile(row):
    for col in ['Car']:
        cell_value = str(row[col])
        for keyword, brands in brands_dict.items():
            if keyword.lower() in cell_value.lower():
                return brands
    return 'Other'

Cars['Brand'] = Cars.apply(assign_profile, axis=1)

In [7]:
brands_counts = Cars['Brand'].value_counts()

print(brands_counts)

Ford                   9993
Chevrolet              8174
Toyota                 7561
Jeep                   4826
GMC                    4162
BMW                    3831
Honda                  3264
Mercedes               3102
Lexus                  2981
Kia                    2808
RAM                    2628
Audi                   2438
Nissan                 2385
Mazda                  1933
Cadillac               1884
Dodge                  1847
Hyundai                1769
Volkswagen             1690
Subaru                 1369
Acura                  1255
Land Rover             1239
Porsche                1043
Volvo                  1018
Genesis                 921
Lincoln                 905
Infiniti                875
Buick                   680
Tesla                   572
Chrysler                539
Mitsubishi              420
Jaguar                  408
Alfa Romeo              234
Maserati                232
MINI                    190
Bentley                 147
Aston Martin        

In [8]:
other_cars = Cars[Cars['Brand'] == 'Other']['Car']

print(other_cars)

Series([], Name: Car, dtype: object)


Perfect!

## Model

The approach is the same as the brand, creating and using a dictionary:

In [9]:
models_df = pd.read_excel('master_data/models.xlsx')

models_df

,keyword,model
0,Outlander,Outlander
1,Mustang,Mustang
2,F-150,F-150
3,Explorer,Explorer
4,Escape,Escape
...,...,...
812,Allante,Allante
813,2,2
814,1,1
815,3,3


In [10]:
models_dict = models_df.set_index('keyword')['model'].to_dict()

def assign_profile(row):
    for col in ['Car']:
        cell_value = str(row[col])
        for keyword, model in models_dict.items():
            if isinstance(keyword, str) and keyword.lower() in cell_value.lower():
                return model
    return 'Other'

Cars['Model'] = Cars.apply(assign_profile, axis=1)

In [11]:
models_counts = Cars['Model'].value_counts()

print(models_counts)

F-150             2491
Silverado         2399
Sierra            1991
Grand Cherokee    1676
Mustang           1507
                  ... 
Vanquish             1
Mazda2               1
S-TYPE               1
G20                  1
924 S                1
Name: Model, Length: 606, dtype: int64


In [12]:
Cars[['Brand','Model','Price_1','Price_2','Mileage']]

,Brand,Model,Price_1,Price_2,Mileage
1,Mazda,CX-5,36703,NaN,"\n\n\n $36,703\n\n MSRP $38,635\n\n\n"
2,Kia,Sportage,28990,NaN,"\n\n\n $28,990\n\n\n\n"
3,Chevrolet,Camaro,41425,NaN,"\n\n\n $41,425\n\n MSRP $41,425\n\n\n"
4,Ford,Bronco,NaN,58900.0,"1,551 mi."
5,Acura,TLX,NaN,34499.0,"30,384 mi."
...,...,...,...,...,...
10494,Chevrolet,Corvette,122455,NaN,"\n\n\n $122,455\n\n\n\n"
10495,Chevrolet,Corvette,125930,NaN,"\n\n\n $125,930\n\n\n\n"
10496,Jeep,Wrangler,53715,NaN,"\n\n\n $53,715\n\n MSRP $58,715\n\n\n ..."
10497,Jeep,Wrangler,91450,NaN,"\n\n\n $91,450\n\n MSRP $86,455\n\n\n ..."


In [13]:
other_cars = Cars[Cars['Model'] == 'Other']['Car']

print(other_cars)

Series([], Name: Car, dtype: object)


Now looks nice!

## Year

In [14]:
Cars['Year'] = Cars['Car'].str[:4]

In [15]:
Cars['Year'].value_counts()

2023    30701
2024    14898
2022     6397
2021     5304
2020     4856
2019     3601
2018     2736
2017     2380
2016     1844
2015     1554
2014     1158
2013      922
2012      670
2011      592
2010      383
2008      329
2007      292
2009      254
2006      219
2005      168
2004      159
2003      100
2002       97
2001       66
2000       52
1999       35
1995       26
1998       22
1997       22
1994       22
1990       16
1996       16
1989       14
1987       11
1993       10
1992        8
1991        7
1982        5
1971        5
1969        5
1973        5
1977        5
1988        4
1986        4
1984        4
1978        4
1979        4
1970        4
1985        3
1972        2
1975        2
1967        2
1974        1
1976        1
1966        1
1959        1
1983        1
1968        1
Name: Year, dtype: int64

Perfect!

## Status

To get the Status, we will transform the Certified ones into a single "Certified" Category:

In [16]:
Cars['Status'].value_counts()

New                        43678
Used                       31436
Toyota Certified             641
Ford Certified               490
Chevrolet Certified          389
BMW Certified                317
Honda Certified              304
Mercedes-Benz Certified      286
Porsche Certified            258
GMC Certified                192
Kia Certified                191
Volkswagen Certified         187
Audi Certified               187
Acura Certified              185
Cadillac Certified           185
Nissan Certified             171
Volvo Certified              163
Jeep Certified               157
INFINITI Certified           126
Hyundai Certified            104
Subaru Certified              78
Lincoln Certified             72
Dodge Certified               62
RAM Certified                 51
Buick Certified               24
Ferrari Certified             19
Chrysler Certified            13
Genesis Certified             10
Bentley Certified              9
Rolls-Royce Certified          9
Maserati C

In [17]:
Cars['Status'] = Cars['Status'].astype(str)
Cars['Status'] = Cars['Status'].apply(lambda x: 'Certified' if 'Certified' in x else x)

In [18]:
Cars['Status'].value_counts()

New          43678
Used         31436
Certified     4891
Name: Status, dtype: int64

## Mileage

We need to transform the Mileage column. First of all we need to delete ones that include the ' mi.' string, and then, for the rest we will delete the ' mi.' string:

In [19]:
def transform_mileage(mileage):
    if isinstance(mileage, str) and ' mi.' in mileage:
        return float(mileage.replace(' mi.', '').replace(',', '').strip())
    else:
        return np.nan

Cars['Mileage'] = Cars['Mileage'].apply(transform_mileage)

In [20]:
Cars

,URL,Car,Status,Price_1,Price_2,Mileage,Dealer_1,Dealer_2,Dealer_3,Brand,Model,Year
1,https://www.cars.com/vehicledetail/5ee74d2b-dd...,2023 Mazda CX-50 2.5 S Premium Plus Package,New,36703,NaN,NaN,NaN,NaN,NaN,Mazda,CX-5,2023
2,https://www.cars.com/vehicledetail/460b9155-fa...,2023 Kia Sportage S,New,28990,NaN,NaN,NaN,Classic Kia,NaN,Kia,Sportage,2023
3,https://www.cars.com/vehicledetail/a0594120-25...,2024 Chevrolet Camaro 2LT,New,41425,NaN,NaN,NaN,Classic Chevrolet Beaumont,NaN,Chevrolet,Camaro,2024
4,https://www.cars.com/vehicledetail/b185d19c-b4...,2023 Ford Bronco Badlands,Used,NaN,58900.0,1551.0,Mike Smith Chrysler Dodge Jeep RAM,NaN,NaN,Ford,Bronco,2023
5,https://www.cars.com/vehicledetail/52bb1768-17...,2021 Acura TLX Advance,Used,NaN,34499.0,30384.0,Mike Smith Nissan,NaN,NaN,Acura,TLX,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
10494,https://www.cars.com/vehicledetail/06665049-49...,2023 Chevrolet Corvette Stingray w/3LT,New,122455,NaN,NaN,NaN,NaN,Chase Chevrolet,Chevrolet,Corvette,2023
10495,https://www.cars.com/vehicledetail/9e367454-56...,2023 Chevrolet Corvette Stingray w/3LT,New,125930,NaN,NaN,NaN,NaN,Chase Chevrolet,Chevrolet,Corvette,2023
10496,https://www.cars.com/vehicledetail/d578331d-53...,2023 Jeep Wrangler Rubicon,New,53715,NaN,NaN,NaN,Stockton Dodge Chrysler Jeep RAM,NaN,Jeep,Wrangler,2023
10497,https://www.cars.com/vehicledetail/3b7dc7d4-60...,2023 Jeep Wrangler Rubicon 392,New,91450,NaN,NaN,NaN,Stockton Dodge Chrysler Jeep RAM,NaN,Jeep,Wrangler,2023


## Dealer

In order to get the Dealer we will combine the three Dealer columns, since just one of them has the right value, and the other ones has NaN:

In [21]:
Cars['Dealer'] = Cars['Dealer_1'].combine_first(Cars['Dealer_2']).combine_first(Cars['Dealer_3'])

In [22]:
Cars

,URL,Car,Status,Price_1,Price_2,Mileage,Dealer_1,Dealer_2,Dealer_3,Brand,Model,Year,Dealer
1,https://www.cars.com/vehicledetail/5ee74d2b-dd...,2023 Mazda CX-50 2.5 S Premium Plus Package,New,36703,NaN,NaN,NaN,NaN,NaN,Mazda,CX-5,2023,NaN
2,https://www.cars.com/vehicledetail/460b9155-fa...,2023 Kia Sportage S,New,28990,NaN,NaN,NaN,Classic Kia,NaN,Kia,Sportage,2023,Classic Kia
3,https://www.cars.com/vehicledetail/a0594120-25...,2024 Chevrolet Camaro 2LT,New,41425,NaN,NaN,NaN,Classic Chevrolet Beaumont,NaN,Chevrolet,Camaro,2024,Classic Chevrolet Beaumont
4,https://www.cars.com/vehicledetail/b185d19c-b4...,2023 Ford Bronco Badlands,Used,NaN,58900.0,1551.0,Mike Smith Chrysler Dodge Jeep RAM,NaN,NaN,Ford,Bronco,2023,Mike Smith Chrysler Dodge Jeep RAM
5,https://www.cars.com/vehicledetail/52bb1768-17...,2021 Acura TLX Advance,Used,NaN,34499.0,30384.0,Mike Smith Nissan,NaN,NaN,Acura,TLX,2021,Mike Smith Nissan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10494,https://www.cars.com/vehicledetail/06665049-49...,2023 Chevrolet Corvette Stingray w/3LT,New,122455,NaN,NaN,NaN,NaN,Chase Chevrolet,Chevrolet,Corvette,2023,Chase Chevrolet
10495,https://www.cars.com/vehicledetail/9e367454-56...,2023 Chevrolet Corvette Stingray w/3LT,New,125930,NaN,NaN,NaN,NaN,Chase Chevrolet,Chevrolet,Corvette,2023,Chase Chevrolet
10496,https://www.cars.com/vehicledetail/d578331d-53...,2023 Jeep Wrangler Rubicon,New,53715,NaN,NaN,NaN,Stockton Dodge Chrysler Jeep RAM,NaN,Jeep,Wrangler,2023,Stockton Dodge Chrysler Jeep RAM
10497,https://www.cars.com/vehicledetail/3b7dc7d4-60...,2023 Jeep Wrangler Rubicon 392,New,91450,NaN,NaN,NaN,Stockton Dodge Chrysler Jeep RAM,NaN,Jeep,Wrangler,2023,Stockton Dodge Chrysler Jeep RAM


## Price

Same approach as Dealer, but we need also to transform into a numeric:

In [23]:
Cars['Price'] = Cars['Price_1'].combine_first(Cars['Price_2'])
Cars['Price'] = pd.to_numeric(Cars['Price'], errors='coerce')

In [24]:
Cars.head(50)

,URL,Car,Status,Price_1,Price_2,Mileage,Dealer_1,Dealer_2,Dealer_3,Brand,Model,Year,Dealer,Price
1,https://www.cars.com/vehicledetail/5ee74d2b-dd...,2023 Mazda CX-50 2.5 S Premium Plus Package,New,36703,NaN,NaN,NaN,NaN,NaN,Mazda,CX-5,2023,NaN,36703.0
2,https://www.cars.com/vehicledetail/460b9155-fa...,2023 Kia Sportage S,New,28990,NaN,NaN,NaN,Classic Kia,NaN,Kia,Sportage,2023,Classic Kia,28990.0
3,https://www.cars.com/vehicledetail/a0594120-25...,2024 Chevrolet Camaro 2LT,New,41425,NaN,NaN,NaN,Classic Chevrolet Beaumont,NaN,Chevrolet,Camaro,2024,Classic Chevrolet Beaumont,41425.0
4,https://www.cars.com/vehicledetail/b185d19c-b4...,2023 Ford Bronco Badlands,Used,NaN,58900.0,1551.0,Mike Smith Chrysler Dodge Jeep RAM,NaN,NaN,Ford,Bronco,2023,Mike Smith Chrysler Dodge Jeep RAM,58900.0
5,https://www.cars.com/vehicledetail/52bb1768-17...,2021 Acura TLX Advance,Used,NaN,34499.0,30384.0,Mike Smith Nissan,NaN,NaN,Acura,TLX,2021,Mike Smith Nissan,34499.0
6,https://www.cars.com/vehicledetail/6c4f8f63-05...,2022 Volkswagen Golf GTI 2.0T Autobahn,Certified,NaN,34000.0,13895.0,Volkswagen of Beaumont,NaN,NaN,Volkswagen,Golf,2022,Volkswagen of Beaumont,34000.0
7,https://www.cars.com/vehicledetail/7dd93af8-87...,2021 GMC Yukon Denali,Used,NaN,56954.0,68506.0,BMW of Beaumont,NaN,NaN,GMC,Yukon,2021,BMW of Beaumont,56954.0
8,https://www.cars.com/vehicledetail/78ee816a-be...,2023 BMW M340 i,New,61715,NaN,NaN,NaN,BMW of Beaumont,NaN,BMW,M340,2023,BMW of Beaumont,61715.0
9,https://www.cars.com/vehicledetail/566bdb25-ca...,2023 Hyundai Sonata Hybrid Limited,New,37945,NaN,NaN,NaN,Hyundai of Silsbee,NaN,Hyundai,Sonata,2023,Hyundai of Silsbee,37945.0
10,https://www.cars.com/vehicledetail/fce328b2-f8...,2023 Hyundai Sonata Hybrid SEL,New,33495,NaN,NaN,NaN,Hyundai of Silsbee,NaN,Hyundai,Sonata,2023,Hyundai of Silsbee,33495.0


# Generating files

Now let's generate the files!

In [25]:
Cars_file = Cars[['Brand','Model','Year','Status','Mileage','Dealer','Price']]
Cars_file.to_excel('cars.xlsx', index=False)
Cars_file.to_csv('cars.csv', index=False, encoding="utf-16")